# TSP and Integer Programming
Based on Lab 10 of ENGRI 1101 Fall 2019

<font color='red'> **Comments in red** </font>

<font color='blue'> **Solutions in blue** </font>

**Objectives:**
- Introduce the students to an integer programming formulation of the traveling salesman problem
- Demonstrate the use of the simplex method to solve two linear relaxations of the traveling salesman problem
- Give students an appreciation of the difficulty of solving optimization problems exactly
- Practice using branch and bound to solve integer programs

**Reading assignment:**
- Handout 7 on Integer Programming and the Traveling Salesman Problem; lecture 19 handout on Branch and Bound by example.

If one is to solve a minimization Integer Program, having lower bounds can help considerably. One way to get a lower bound is to use an LP relaxation of the problem. For the TSP, we first construct a correct integer programming formulation of the problem (see, e.g., the prelab formulation), and then obtain the LP relaxation by dropping the integrality constraints.

**Q1:** Why does the optimal solution to the LP relaxation provide a lower bound on the optimal solution to the TSP?

**A:** <font color='blue'> The optimal solution to the LP relaxation provides a lower bound because the IP is the same problem except with an added constraint (integrality) which can only remove feasible solutions. Hence, the optimal solution can only decrease.</font>

Using an LP relaxation is not the only possible way of getting a lower bound for a problem. For example, another lower bound for the TSP is the cost of the minimum spanning tree on the same graph. Complete the following argument that this is the case: If you start with a tour, and delete one edge, the remaining graph is connected, i.e., there is a path between any two nodes in the graph.

**Q2:** Why does this imply that the cost of a minimum spanning tree is a lower bound on the cost of an optimal tour?

**A:** <font color='blue'> Consider an arbitrary tour of the nodes. If we remove one edge, we are left with a spanning tree. By definition, this spanning tree must cost the same or more than the minimum spanning tree. Furthermore, the tour must cost more than this spanning tree since it has that additional edge. Hence, any tour must have cost no less than the cost of the minimum spanning tree. </font>

In the prelab, you were asked to show that the minimum cost of a fractional 2-matching is a lower bound on the optimal tour length. Now consider an instance of the TSP on 6 nodes, where the distance between node $i$ and $j$ are given by the ($i,j$)-th entry in the following matrix.

|   |   |   |   |   |   |
|---|---|---|---|---|---|
| - | 1 | 1 | 5 | 6 | 6 |
| 1 | - | 1 | 6 | 5 | 6 |
| 1 | 1 | - | 6 | 6 | 5 |
| 5 | 6 | 6 | - | 1 | 1 |
| 6 | 5 | 6 | 1 | - | 1 |
| 6 | 6 | 5 | 1 | 1 | - |

We can also depict this instance graphically as follows:

![title](graph1.png)

The distance or cost of traveling between node i and j corresponds to the shortest path from $i$ to $j$ in the graph above. For example $c_{16} = 6$ and the shortest route between nodes 1 and 6 is of length 6: 1 → 4 → 6.

First, let’s consider the lower bound that we get from calculating an optimal solution to the Minimum Spanning Tree Problem for this graph. Note that there we do not have to consider the edges that are not drawn in the graph. 

**Q3:** Draw an optimal solution for the Minimum Spanning Tree Problem for the instance, and give the length of the solution.

**A:** <font color='blue'> The length of the optimal minimum spanning tree (not unique) is 9. </font>

![title](graph1_mst.png)

We will now generate a second lower bound for this instance by solving the fractional 2-matching LP that corresponds to this instance. To solve the corresponding 2-matching LP, we will use OR-Tools again. Note that there are no integrality constraints, so this is indeed an LP, i.e., the fractional 2-matching LP. Run the model cell.

In [1]:
# data
numcities = 6
cost = {(1,2) : 1, (1,3) : 1, (1,4) : 5, (1,5) : 6, (1,6) : 6,
                   (2,3) : 1, (2,4) : 6, (2,5) : 5, (2,6) : 6,
                              (3,4) : 6, (3,5) : 6, (3,6) : 5,
                                         (4,5) : 1, (4,6) : 1,
                                                    (5,6) : 1}
cuts = [[1]]

In [2]:
# imports
from ortools.linear_solver import pywraplp as OR
import itertools  

# 2-matching relaxation for TSP
# with template for adding subtour elimination constraints
def Two_Matching(numcities, cost, cuts):
    
    CITIES = list(range(1,numcities+1))
    EDGES = [(i,j) for (i,j) in itertools.product(CITIES, CITIES) if i < j]
    
    # define model
    mod = OR.Solver('2matching', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {(i,j) : mod.NumVar(0, 1, ('(%s,%s)' % (i,j))) for (i,j) in EDGES}
    
    # minimize total cost
    mod.Minimize(sum(cost[i,j]*x[i,j] for (i,j) in EDGES))
    
    # subject to degree 2
    for k in CITIES:
        mod.Add(sum(x[i,j] for (i,j) in EDGES if i == k) + 
                sum(x[i,j] for (i,j) in EDGES if j == k) == 2)
    
    # subject to cuts
    for S in cuts:
        mod.Add(sum(x[i,j] for (i,j) in EDGES if (i in S and j not in S) or 
                                                 (j in S and i not in S)) >= 2)
        
    # solve and print solution
    mod.Solve()
    print('Objective =', mod.Objective().Value())
    print('Solution:')
    for v in mod.variables():
        print(v.name(),':', v.solution_value())

In [3]:
Two_Matching(numcities, cost, cuts)

Objective = 6.0
Solution:
(1,2) : 1.0
(1,3) : 1.0
(1,4) : 0.0
(1,5) : 0.0
(1,6) : 0.0
(2,3) : 1.0
(2,4) : 0.0
(2,5) : 0.0
(2,6) : 0.0
(3,4) : 0.0
(3,5) : 0.0
(3,6) : 0.0
(4,5) : 1.0
(4,6) : 1.0
(5,6) : 1.0


**Q4:** Report the results.

**A:** <font color='blue'> The objective value is 6 and we got an integral solution.</font>

**Q5:** Is it possible that if we were to solve the minimum-cost 2-matching problem (where we add the integrality constraints to the above LP) for this data set we would get a better lower bound? Why or why not?

**A:** <font color='blue'> This is already a feasible solution subject to the integer constraint. Since this is a relaxation, the more constrained integer solution can't be better. Hence, we can not get a better lower bound. </font>

**Q6:** Did we get a feasible tour? 

**A:** <font color='blue'> No, we have two separate sub-tours. </font>

A third lower bound for the TSP can be obtained from the LP relaxation of an IP formulation of the TSP. In particular, we will be interested in the following linear programming problem, which is a relaxation of an IP for the TSP:

$$(1)\quad \text{minimize} \sum_{i=1}^{n-1}\sum_{j = i+1}^n c_{ij}x_{ij}$$
$$\text{subject to}$$
$$(2)\quad \sum_{j=1}^{i-1} x_{ji} + \sum_{j = i+1}^n x_{ij} = 2, \forall i = 1,\dots,n$$
$$(3)\quad \sum_{\{i,j\} \in \delta(S)}x_{ij} \geq 2, \forall S \subset V$$
$$(4)\quad 0 \leq x_{ij} \leq 1, \forall i = 1,\dots,n-1, j = i+1,\dots,n$$

This linear program is called the Subtour Elimination LP (recall that with the integrality constraints the Subtour Elimination LP is a correct integer program for the TSP problem).

**Q7:** Is the optimal solution to the fractional 2-matching problem that you just found a feasible solution to this linear program? Why or why not?

**A:** <font color='blue'> No, because if $S = \{1,2,3\}$, then there are not at least 2 edges spanning between {1,2,3} and {4,5,6}. Hence, the subtour elimination constraint is violated.</font>

One problem with the subtour elimination constraints is that we add a constraint of the form (3) for each set of the vertices S (disregarding the set of no vertices S = {} and the set S of all vertices S = V , since the subtour elimination constraints don’t make sense). 

**Q8:** If we have n = 10 vertices, how many subtour elimination constraints would we need to write? What if n = 100?

**A:** <font color='blue'> If we have $n = 10$ vertices, we would need $2^{10} - 2 = 1022$ constriants because that is the number of subsets of $V$ when $|V| = 10$ excluding the empty set and $V$. If $n = 100$, we would have $2^{100} - 2$ constraints</font>

For $n = 100$ we’d need about $2^{100} − 2$ constraints, which is known technically as “way too many.” (You can do a little bit better by being clever, but not substantially). In practice, we need a method to add subtour elimination constraints one at a time, without writing them all down. In fact, one can use a maximum flow algorithm to automatically check if the constraints (3) are all satisfied, and if not, find a violated constraint, without having to check them all, one at a time (we are not going to explain how this can be done). This leads to the following algorithm to solve this very large LP. Solve the minimum-cost fractional 2-matching problem (using the simplex method). Check if this solution satisfies all of the constraints (3). If yes, then this is an optimal solution to the Subtour Elimination LP.

**Q9:** Why?

**A:** <font color='blue'> The minimum-cost fractional 2-matching problem is the same LP as the subtour elimination LP minus constraints (3) so if the optimal solution satifies constraints (3) then it will clearly be optimal to subtour elimination since adding a constraint can only increase the optimal value.</font>

If not, then the algorithm will find a constraint that is violated. This constraint is added to the fractional 2-matching LP, and the algorithm solves this new LP. Now for this new solution, check again if all of the constraints (3) are satisfied. If yes, then you have the optimal solution to the Subtour Elimination LP. Otherwise continue this procedure of adding violated constraints until an optimal solution is found.

Previously, we found that the subtour elimination constraints were not satisfied, so our current solution (the one you most recently drew, by solving the fractional 2-matching LP without any subtour constraints) is not feasible for the subtour LP. Hence, we have to find and add a violated subtour constraint corresponding to a set of vertice S. You already found this at the end of page 4!

**Q10:** Write down this set S.

**A:** <font color='blue'> S = \{1,2,3\}</font>

You will now add this constraint to your LP. This can be done by updating ```cuts```. This variable contains a list of cuts. Right now, the only cut it contains is $S = \{1\}$. Let's update it to include the cut you found. For example, if the cut you found was $S = \{1,2,5\}$, you would update ```cuts``` as follows:

In [4]:
# cuts = [[1],[1,2,5]] 
cuts = [[1],[1,2,3]] 

Solve the updated LP. 

In [5]:
Two_Matching(numcities, cost, cuts)

Objective = 14.0
Solution:
(1,2) : 0.0
(1,3) : 1.0
(1,4) : 1.0
(1,5) : 0.0
(1,6) : 0.0
(2,3) : 1.0
(2,4) : 0.0
(2,5) : 1.0
(2,6) : 0.0
(3,4) : 0.0
(3,5) : 0.0
(3,6) : 0.0
(4,5) : 0.0
(4,6) : 1.0
(5,6) : 1.0


**Q11:** Give the objective value of the new optimal solution and draw the graph with this solution.

**A:** <font color='blue'> The objective value is 14.</font>

![title](graph1_tour.png)

If you can find any additional violated subtour constraints, repeat the above. When you are done, you have found an optimal solution to the Subtour Elimination LP linear program, even though you didn’t have to write out all sub-tour elimination constraints explicitly!


**Q12:** Draw your solution to the subtour LP and give the objective value.

**A:** **A:** <font color='blue'> The optimal value is 14.</font>

![title](graph1_tour.png)